In [22]:
%load_ext autoreload
%autoreload 2

import os as os
import numpy as np

import parmed as pmd
import pytraj as pt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import logging
import datetime as dt
d_date = dt.datetime.now()
logging.basicConfig(format='%(asctime)s %(message)s', datefmt='%Y-%m-%d %I:%M:%S %p',
                    level=logging.DEBUG)
logging.info('Started logging...')

import paprika
print(paprika.__version__)

from paprika.restraints import static_DAT_restraint
from paprika.restraints import DAT_restraint
from paprika.restraints import amber_restraint_line
from paprika.restraints import create_window_list

from paprika.utils import make_window_dirs

from paprika.analysis import fe_calc

2018-05-30 07:06:11 PM Started logging...


0.0.3-with-release-tweaks


In [24]:
structure = pmd.load_file('systems/a-bam-p/confirm-original/p045/full.hmr.topo',
                          'systems/a-bam-p/confirm-original/p045/full.crds', 
                          structure=True)

In [25]:
structure

<AmberParm 6159 atoms; 2023 residues; 6148 bonds; PBC (orthogonal); parametrized>

In [26]:
structure.strip(':BAM')

In [27]:
structure.save('systems/a-bam-p/confirm-original/full.hmr.noligand.prmtop')

OSError: systems/a-bam-p/confirm-original/full.hmr.noligand.prmtop exists; not overwriting

> /home/dslochower/data/applications/anaconda3/envs/smirnoff-paprika/lib/python3.6/site-packages/parmed/structure.py(1478)save()
   1476         if not hasattr(fname, 'write'):
   1477             if os.path.exists(fname) and not overwrite:
-> 1478                 raise IOError('%s exists; not overwriting' % fname)
   1479         else:
   1480             if format is None:

ipdb> 


In [28]:
structure.save('systems/a-bam-p/confirm-original/full.hmr.noligand.inpcrd')

OSError: systems/a-bam-p/confirm-original/full.hmr.noligand.inpcrd exists; not overwriting

> /home/dslochower/data/applications/anaconda3/envs/smirnoff-paprika/lib/python3.6/site-packages/parmed/structure.py(1478)save()
   1476         if not hasattr(fname, 'write'):
   1477             if os.path.exists(fname) and not overwrite:
-> 1478                 raise IOError('%s exists; not overwriting' % fname)
   1479         else:
   1480             if format is None:

ipdb> 


# Setup `attach`

In [29]:
hg = pmd.load_file('systems/a-bam-p/confirm-original/full.hmr.noligand.prmtop',
                   'systems/a-bam-p/confirm-original/full.hmr.noligand.inpcrd',
                    structure=True)

dummy_anchors = [':1', ':2', ':3']
host_anchors  = [':4@O3', ':6@C1', ':8@C6']

In [30]:
attach_string = '0.00 0.40 0.80 1.60 2.40 4.00 5.50 8.65 11.80 18.10 24.40 37.00 49.60 74.80 100.00'
attach_fractions = [float(i) / 100 for i in attach_string.split()]

windows = [len(attach_fractions), 0, 0]
print(f'There are {windows} windows in this attach-pull-release calculation.')

There are [15, 0, 0] windows in this attach-pull-release calculation.


In [31]:
static_restraint_atoms = [[dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[2], dummy_anchors[1], dummy_anchors[0], host_anchors[0]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[1], dummy_anchors[0], host_anchors[0], host_anchors[1]],
                          [dummy_anchors[0], host_anchors[0], host_anchors[1], host_anchors[2]],
                        ]

static_restraint_distance_fc = 5.0
static_restraint_angle_fc = 100.0

In [32]:
host_conformational_template = [['O5', 'C1', 'O1', 'C4'],
                                ['C1', 'O1', 'C4', 'C5']
                               ]

host_residues = len(hg[':MGO'].residues)
first_host_residue = hg[':MGO'].residues[0].number + 1
conformational_restraint_atoms = []
conformational_restraint_targets = []
conformational_restraint_fc = 6.0

for n in range(first_host_residue, host_residues + first_host_residue):
    if n + 1 < host_residues + first_host_residue:
        next_residue = n + 1
    else:
        next_residue = first_host_residue
    conformational_restraint_atoms.append(
        [f':{n}@{host_conformational_template[0][0]}',
         f':{n}@{host_conformational_template[0][1]}',
         f':{n}@{host_conformational_template[0][2]}',
         f':{next_residue}@{host_conformational_template[0][3]}'
        ])
    conformational_restraint_targets.append(104.30)
    conformational_restraint_atoms.append(
        [f':{n}@{host_conformational_template[1][0]}',
         f':{n}@{host_conformational_template[1][1]}',
         f':{next_residue}@{host_conformational_template[1][2]}',
         f':{next_residue}@{host_conformational_template[1][3]}'
        ])
    conformational_restraint_targets.append(-108.8)

In [33]:
static_restraints = []
for index, atoms in enumerate(static_restraint_atoms):    
    this = static_DAT_restraint(restraint_mask_list=atoms,
                        num_window_list=windows, 
                        ref_structure=hg,
                        force_constant=static_restraint_angle_fc if len(atoms) > 2 else static_restraint_distance_fc,
                        amber_index=True,
                        continuous_apr=False)
    static_restraints.append(this)

2018-05-30 07:06:39 PM Calculating attach targets and force constants...
2018-05-30 07:06:39 PM Attach, Method #1
2018-05-30 07:06:39 PM Calculating pull targets and force constants...
2018-05-30 07:06:39 PM No restraint info set for the pull phase! Skipping...
2018-05-30 07:06:39 PM Calculating release targets and force constants...
2018-05-30 07:06:39 PM No restraint info set for the release phase! Skipping...
2018-05-30 07:06:39 PM Number of attach windows = 15
2018-05-30 07:06:39 PM This restraint will be skipped in the pull phase
2018-05-30 07:06:39 PM This restraint will be skipped in the release phase
2018-05-30 07:06:39 PM Assigning atom indices...
2018-05-30 07:06:39 PM There are 1 atoms in the mask :1  ...
2018-05-30 07:06:39 PM There are 1 atoms in the mask :4@O3  ...
2018-05-30 07:06:39 PM Calculating attach targets and force constants...
2018-05-30 07:06:39 PM Attach, Method #1
2018-05-30 07:06:39 PM Calculating pull targets and force constants...
2018-05-30 07:06:39 PM No

In [38]:
conformational_restraints = []
for index, atoms in enumerate(conformational_restraint_atoms): 
    
    this = DAT_restraint()
    this.auto_apr = True
    this.amber_index = True
    this.topology = hg
    this.mask1 = atoms[0]
    this.mask2 = atoms[1]
    this.mask3 = atoms[2]
    this.mask4 = atoms[3]
    
    this.attach['fraction_list'] = attach_fractions
    this.attach['target'] = conformational_restraint_targets[index]
    this.attach['fc_final'] = conformational_restraint_fc
    
    this.continuous_apr = False
    
    this.initialize()
    conformational_restraints.append(this)

2018-05-30 07:13:31 PM Calculating attach targets and force constants...
2018-05-30 07:13:31 PM Attach, Method #3
2018-05-30 07:13:31 PM Calculating pull targets and force constants...
2018-05-30 07:13:31 PM No restraint info set for the pull phase! Skipping...
2018-05-30 07:13:31 PM Calculating release targets and force constants...
2018-05-30 07:13:31 PM No restraint info set for the release phase! Skipping...
2018-05-30 07:13:31 PM Number of attach windows = 15
2018-05-30 07:13:31 PM This restraint will be skipped in the pull phase
2018-05-30 07:13:31 PM This restraint will be skipped in the release phase
2018-05-30 07:13:31 PM Assigning atom indices...
2018-05-30 07:13:31 PM There are 1 atoms in the mask :4@O5  ...
2018-05-30 07:13:31 PM There are 1 atoms in the mask :4@C1  ...
2018-05-30 07:13:31 PM There are 1 atoms in the mask :4@O1  ...
2018-05-30 07:13:31 PM There are 1 atoms in the mask :5@C4  ...
2018-05-30 07:13:31 PM Calculating attach targets and force constants...
2018-0

2018-05-30 07:13:32 PM There are 1 atoms in the mask :9@C4  ...
2018-05-30 07:13:32 PM Calculating attach targets and force constants...
2018-05-30 07:13:32 PM Attach, Method #3
2018-05-30 07:13:32 PM Calculating pull targets and force constants...
2018-05-30 07:13:32 PM No restraint info set for the pull phase! Skipping...
2018-05-30 07:13:32 PM Calculating release targets and force constants...
2018-05-30 07:13:32 PM No restraint info set for the release phase! Skipping...
2018-05-30 07:13:32 PM Number of attach windows = 15
2018-05-30 07:13:32 PM This restraint will be skipped in the pull phase
2018-05-30 07:13:32 PM This restraint will be skipped in the release phase
2018-05-30 07:13:32 PM Assigning atom indices...
2018-05-30 07:13:32 PM There are 1 atoms in the mask :8@C1  ...
2018-05-30 07:13:32 PM There are 1 atoms in the mask :8@O1  ...
2018-05-30 07:13:32 PM There are 1 atoms in the mask :9@C4  ...
2018-05-30 07:13:32 PM There are 1 atoms in the mask :9@C5  ...
2018-05-30 07:1

In [39]:
window_list = create_window_list(conformational_restraints)

2018-05-30 07:13:33 PM All restraints are not "continuous_apr" style.
2018-05-30 07:13:33 PM Restraints appear to be consistent


In [41]:
window_list

['a000',
 'a001',
 'a002',
 'a003',
 'a004',
 'a005',
 'a006',
 'a007',
 'a008',
 'a009',
 'a010',
 'a011',
 'a012',
 'a013',
 'a014']

In [42]:
prefix = 'systems/a-bam-p/confirm-original/no-ligand/'

In [45]:
for window in window_list:
    try:
        os.makedirs(os.path.join(prefix, window))
        # os.remove(os.path.join(prefix, window, 'full.hmr.noligand.prmtop'))
        # os.remove(os.path.join(prefix, window, 'full.hmr.noligand.inpcrd'))
        os.symlink(os.path.join('../../', 'full.hmr.noligand.prmtop'), os.path.join(prefix, window, 'full.hmr.noligand.prmtop'))
        os.symlink(os.path.join('../../', 'full.hmr.noligand.inpcrd'), os.path.join(prefix, window, 'full.hmr.noligand.inpcrd'))
    except:
        pass

In [46]:
for window in window_list:
    with open(os.path.join(prefix, window, 'disang.rest'), 'w') as file:
        for restraint in static_restraints + conformational_restraints:
            
            try:
                string = amber_restraint_line(restraint, 'attach', int(window[1:]))
            except:
                # Hack to get around trying to access
                # --> 630         'r2': restraint.phase[phase]['targets'][window],
                # for wall restraints, during pull.
                continue
            if string is not None:
                file.write(string)

In [50]:
%pdb
structure = pt.load(
    os.path.join('/home', 'dslochower', 'kirkwood', 'projects', 'smirnoff-host-guest-simulations', 
                            'systems', 'a-bam-p', 'confirm-original', 'no-ligand', 'full.hmr.noligand.inpcrd'),
    os.path.join('/home', 'dslochower', 'kirkwood', 'projects', 'smirnoff-host-guest-simulations', 
                            'systems', 'a-bam-p', 'confirm-original', 'no-ligand', 'full.hmr.noligand.prmtop'))

stripped = structure.strip(':WAT,:Na+,:Cl-')

analyze = fe_calc()
analyze.prmtop = stripped.topology
analyze.trajectory = 'prod.*.nc'
analyze.path = os.path.join('/home', 'dslochower', 'kirkwood', 'projects', 'smirnoff-host-guest-simulations', 
                            'systems', 'a-bam-p', 'confirm-original', 'no-ligand')


analyze.restraint_list = conformational_restraints
analyze.collect_data()
analyze.methods = ['ti-block']
analyze.quicker_ti_matrix = True
analyze.bootcycles = 1
analyze.compute_free_energy()



2018-05-31 07:13:20 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a000...
2018-05-31 07:13:21 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a000...
2018-05-31 07:13:21 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a000...
2018-05-31 07:13:21 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a000...


Automatic pdb calling has been turned OFF


2018-05-31 07:13:21 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a000...
2018-05-31 07:13:21 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a000...
2018-05-31 07:13:21 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a000...
2018-05-31 07:13:21 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a000...
2018-05-31 07:13:21 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a000...
2018-05-31 07:13:21 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/

2018-05-31 07:13:22 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a004...
2018-05-31 07:13:22 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a004...
2018-05-31 07:13:22 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a004...
2018-05-31 07:13:22 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a004...
2018-05-31 07:13:22 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a004...
2018-05-31 07:13:23 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/

2018-05-31 07:13:24 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a008...
2018-05-31 07:13:24 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a008...
2018-05-31 07:13:24 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a008...
2018-05-31 07:13:24 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a008...
2018-05-31 07:13:24 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a008...
2018-05-31 07:13:24 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/

2018-05-31 07:13:26 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a012...
2018-05-31 07:13:26 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a012...
2018-05-31 07:13:26 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a012...
2018-05-31 07:13:26 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a012...
2018-05-31 07:13:26 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/a012...
2018-05-31 07:13:26 AM Reading restraint data for /home/dslochower/kirkwood/projects/smirnoff-host-guest-simulations/systems/a-bam-p/confirm-original/no-ligand/

[ 41.56531265  42.05442606  42.53958784 ...,  63.79844873  63.68913581
  63.57138963]
[  0.00000000e+00   4.00000000e-05   8.00000000e-05 ...,   3.88000000e-03
   3.92000000e-03   3.96000000e-03]
[ 41.56531265  42.05442606  42.53958784 ...,  31.0448779   30.97153862
  30.91313338]
[  0.00000000e+00   4.00000000e-05   8.00000000e-05 ...,   7.88000000e-03
   7.92000000e-03   7.96000000e-03]
[ 41.56531265  42.05442606  42.53958784 ...,  26.85566635  26.77528754
  26.69269338]
[  0.00000000e+00   4.00000000e-05   8.00000000e-05 ...,   1.57600000e-02
   1.58400000e-02   1.59200000e-02]
[ 41.56531265  42.05442606  42.53958784 ...,  17.18850611  17.13473827
  17.0829798 ]
[  0.00000000e+00   4.00000000e-05   8.00000000e-05 ...,   2.37600000e-02
   2.38400000e-02   2.39200000e-02]
[ 41.56531265  42.05442606  42.53958784 ...,  25.37324533  25.34308605
  25.3043434 ]
[  0.00000000e+00   4.00000000e-05   8.00000000e-05 ...,   3.95200000e-02
   3.96800000e-02   3.98400000e-02]
[ 41.56531265  42.05

In [51]:
analyze.results

{'attach': {'ti-block': {'fe': 7.3545865761794946,
   'sem': 0.0,
   'fe_matrix': array([[ 0.        ,  0.22819704,  0.41496523, ...,  5.57953764,
            6.62757863,  7.35458658],
          [-0.22819704,  0.        ,  0.        , ...,  0.        ,
            0.        ,  7.12384921],
          [-0.41496523, -0.        ,  0.        , ...,  0.        ,
            0.        ,  6.93838568],
          ..., 
          [-5.57953764, -0.        , -0.        , ...,  0.        ,
            0.        ,  1.76868443],
          [-6.62757863, -0.        , -0.        , ..., -0.        ,
            0.        ,  0.71856542],
          [-7.35458658, -7.12384921, -6.93838568, ..., -1.76868443,
           -0.71856542,  0.        ]]),
   'sem_matrix': array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          ..., 
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
          [ 0.,  0.,  0., ...,  0.,  0.,  0.],
  